# 1. Install all dependencies

Install the dependencies using the lsi.yml file provided in the repository.

# 2. Import all necessary libraries

In [ ]:
import torch
import numpy as np
from PIL import Image
from options.train_options import TrainOptions
from models.pSp_o import pSp_o
import torchvision.transforms as trans
import glob
import facer
import os
from utils.metrics_downstream import plot_land_overlay, plot_segmentation_overlay, Metrics, load_attributes
import torch.nn.functional as F

#Image transforms function
image_transforms = trans.Compose([
        trans.Resize((256, 256)),
		trans.ToTensor(),
		trans.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
	])

#Retrieve default args
opts = TrainOptions()
opts, unknown = opts.parser.parse_known_args()

# Download the pre-trained models and checkpoints

The models and data can be downloaded at: (https://drive.google.com/drive/folders/11g9t4ZLKhMfigPRa7dLdqmp5vlThFgoB?usp=drive_link)


Place them inside their respective folders in the main directory.
- ./paper_checkpoints
    - Contains the checkpoints for the paper models
- ./pretrained_models
    - Pre-trained models used in loss functions and the generative models themselves
- ./datasets/images and datasets/real/
    - Input images and experimental measurements used on paper

## Compression Ratio

In [ ]:
# Real will load the physically measured intermediate latents.
opts.ini_w = "real"

compression_dict = {
    "1:512": 128, # 1x128 measurements
    "1:2048": 32, # 1x32 measurements
}

In [ ]:
n_scalars = compression_dict["1:512"]

## Loading Models and Configuration

In [ ]:
opts.device = 'cuda:0'
opts.cls = True
opts.segment = True
opts.landmarks = True
opts.segment_fts = True 
opts.encoding_size =  n_scalars


In [ ]:
if n_scalars == 128:
    opts.checkpoint_path = "paper_checkpoints/real_128.pt"
    path = "datasets/real/128/"
elif n_scalars == 32:
    opts.checkpoint_path = "paper_checkpoints/real_32.pt"
    path = "datasets/real/32/"
else:
    raise ValueError(f"Invalid compression ratio: {n_scalars}")
images = sorted(glob.glob(path+"*.npz"))
print("Number of images:",len(images))

In [ ]:
print("\n=== Loading model components ===")
net = pSp_o(opts).to(opts.device)
net = net.to(opts.device)

# Evaluation

### Create Directory

In [ ]:
path_out = f"out/real/{n_scalars}/"

if os.path.exists(path_out):
		pass
else:
    os.makedirs(path_out)

### Save Result

In [ ]:
net.eval()
with torch.no_grad():
    for i,image in enumerate(images):
        name = (image.split("/")[-1])

        # Load read measurement
        image = np.load(image)
        image = torch.from_numpy(image['array']).to(opts.device)
        out, test, latent_w_plus = net.forward(image.unsqueeze(0),return_latents=True, global_step=0)
    
        # Load GT image
        image = Image.open("datasets/images/"+name[:-4]+".jpg")
        image = image_transforms(image).unsqueeze(0).to(opts.device)
        
        ## Landmarks Evaluation
        plot_land_overlay(torch.zeros_like(out)-1, test[2][1], name=path_out+name[:-4]+"_land.png",new_logits=None,alpha=0.2)
        
        #Segmentation Evaluation
        plot_segmentation_overlay(torch.zeros_like(image)-1, test[2][0].detach().to("cpu"), name=path_out+name[:-4]+"_seg.png", logits_f=True, alpha=1.0)
        
        #Reconstruction
        out = (((out*0.5)+0.5)*255).clamp(0,255)
        out = out.squeeze(0).to("cpu").permute(1,2,0).detach().numpy().astype(np.uint8)
        Image.fromarray(out).save(path_out+name[:-4]+"_rec.png")

### Calculate Metrics

### Loading ground truth attributes and models

In [ ]:
attributes_gt = load_attributes("datasets/list_attr_celeba.txt", images)
metrics = Metrics(19, ['background', 'neck', 'face', 'cloth', 'rr', 'lr', 'rb', 'lb', 're', 'le', 'nose', 'imouth', 'llip', 'ulip', 'hair', 'eyeg', 'hat', 'earr', 'neck_l'])
metrics.reset()

face_detector = facer.face_detector('retinaface/mobilenet', device="cuda:0")
face_landmark = facer.face_aligner('farl/ibug300w/448', device="cuda:0")
face_parser = facer.face_parser('farl/celebm/448', device="cuda:0")
face_attr = facer.face_attr("farl/celeba/224", device="cuda")

In [ ]:
net.eval()
with open(path_out+"output.txt", "w") as file:
    file.write("Name, D, L2, NME \n")
    d1 = 0
    nme1 = 0
    l21 = 0
    total = 0
    accuracy_t = 0
    with torch.no_grad():
        for i,image in enumerate(images):
            name = (image.split("/")[-1])

            # Load read measurement
            image = np.load(image)
            image = torch.from_numpy(image['array']).to(opts.device)
            out, test, latent_w_plus = net.forward(image.unsqueeze(0),return_latents=True, global_step=0)
        
            # Load GT image
            image = Image.open("datasets/images/"+name[:-4]+".jpg")
            image = image_transforms(image).unsqueeze(0).to(opts.device)
            
            ## Landmarks Evaluation
            input_gt = ((image.detach() + 1) / 2 * 255).clamp(0, 255).to(torch.uint8)
            faces = face_detector(input_gt)
            gt = ((face_landmark(input_gt,faces)['alignment'])[0])#/255.0
            x_min, _ = torch.min(gt[:, 0], dim=0)
            y_min, _ = torch.min(gt[:, 1], dim=0)
            x_max, _ = torch.max(gt[:, 0], dim=0)
            y_max, _ = torch.max(gt[:, 1], dim=0)
            # Calculate the normalization factor d (bounding box diagonal)
            d = torch.sqrt((x_max - x_min) ** 2 + (y_max - y_min) ** 2)

            # Calculate the Euclidean distances between corresponding landmarks
            distances = torch.sqrt(torch.sum((test[2][1][0]*255 - gt) ** 2, dim=1))
            l2 = distances.mean()
            nme = ((distances/d).mean())*100 #100 is arbitrary, just for visualization purposes.
            d1 += d
            nme1 += nme
            l21 += l2
            total+=1
            
            #Segmentation Evaluation
            gt_seg = ((face_parser(input_gt,faces)['seg']['logits']))
            gt_seg = gt_seg.argmax(dim=1).float()
            y_pred = torch.argmax(test[2][0], dim=1) # get the most likely prediction
            metrics.add_batch(gt_seg.detach().cpu().numpy(), y_pred.detach().cpu().numpy())

            #Attributes Evaluation
            att = (attributes_gt[i]+1)/2
            correct_predictions = torch.sum(torch.round(F.sigmoid(test[2][2])).to("cpu") == att,1)
            accuracy = ((correct_predictions / 40) * 100)
            accuracy_t += accuracy
 
        file.write("{}, {}, {}, {}\n".format("Landmarks Metrics:", d1/total, l21/total, nme1/total))  
        file.write("{}{}\n".format("Segmentation Metrics:\n",metrics.get_table()))  
        file.write("{}, {}\n".format("Accuracy Classification:",(accuracy_t/total).item()))